In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [2]:
# 1. Add the clean movie function that takes in the argument, "movie".

def clean_movie(movie):

    
    movie = dict(movie)
    
    alt_titles={}
    
    # now loop
    for key in ['Also known as','Arabice','Cantonese','Chinese','French','Hangul','Hebrew','Hepburn','Japanese'
               ,'Literally','Mandarin','McCune-Reischauer','Original title','Polish','Revised Romanization','Romanized'
                ,'Russian','Simplified','Traditional','Yiddish']:
        
        if key in movie:
            
            alt_titles[key]=movie[key]
            
            movie.pop(key)
    
    if len(alt_titles) > 0:
        movie['alt_titles']=alt_titles
 
    def change_column_name(old_name,new_name):
        if old_name in movie:
            movie[new_name]=movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
    

    
    return(movie)

In [3]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load():
    
    kaggle_metadata=pd.read_csv('Resources/movies_metadata.csv',low_memory=False)
    ratings=pd.read_csv('Resources/ratings.csv')

    
    with open('Resources/wikipedia.movies.json', mode='r') as file:
        wiki_movies_raw=json.load(file)
    
    # 3. Write a list comprehension to filter out TV shows.
    wiki_tv_removal=[movie for movie in wiki_movies_raw if 'No. of episodes' not in movie]
    
    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    wiki_clean = [clean_movie(movie) for movie in wiki_tv_removal] 
                 
     # 5. Read in the cleaned movies list from Step 4 as a DataFrame. 
    wiki_clean_df = pd.DataFrame(wiki_clean)

    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    wiki_movies_df['imdb_id'] = []

    # Print the beginning of the logging.
    print("Beginning IMDB_ID Parsing    ")
    print("-----------------------------")
    
    #Loops through all IMDB_Links
    for i, link in enumerate(wiki_movies_df['imdb_link']):
        
        
        
    print(f"Processing IMDB_Link: {wiki_movies_df[imdb_link]})
    
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
        
        
    except:
        print(f"{wiki_movies_df['imdb_id']} is a duplicate. Skipping...")
        pass 

    return wiki_movies_df, kaggle_metadata, ratings 

In [4]:
extract_transform_load()

(                                                    url    year  \
 0     https://en.wikipedia.org/wiki/The_Adventures_o...  1990.0   
 1     https://en.wikipedia.org/wiki/After_Dark,_My_S...  1990.0   
 2      https://en.wikipedia.org/wiki/Air_America_(film)  1990.0   
 3       https://en.wikipedia.org/wiki/Alice_(1990_film)  1990.0   
 4         https://en.wikipedia.org/wiki/Almost_an_Angel  1990.0   
 ...                                                 ...     ...   
 7300    https://en.wikipedia.org/wiki/Holmes_%26_Watson  2018.0   
 7301     https://en.wikipedia.org/wiki/Vice_(2018_film)  2018.0   
 7302  https://en.wikipedia.org/wiki/On_the_Basis_of_Sex  2018.0   
 7303  https://en.wikipedia.org/wiki/Destroyer_(2018_...  2018.0   
 7304  https://en.wikipedia.org/wiki/Black_Mirror:_Ba...  2018.0   
 
                                   imdb_link                            title  \
 0     https://www.imdb.com/title/tt0098987/  The Adventures of Ford Fairlane   
 1     https://www.i